In [ ]:
from pathlib import Path
import xarray as xr
from datetime import datetime
import pandas as pd
import plotly.express as px

# 1. Loading the data

In [ ]:
DATA_PATH = Path("data")
COPERNICUS_DATA_PATH = DATA_PATH / "COPERNICUS"

folder_name = "historical"
file_name = "preprocessed.nc"

In [ ]:
raw_data = xr.load_dataset(COPERNICUS_DATA_PATH / folder_name / file_name)

In [ ]:
rainfall = raw_data["RR"]

# 2. Exploration

In [ ]:
dunkerque_coords = (51.0413242745524, 2.3725602487522712)

dunkerque_rainfall = rainfall.sel(
    lat=dunkerque_coords[0], lon=dunkerque_coords[1], method="nearest", drop=True
)

In [ ]:
dunkerque_rainfall.plot()

In [ ]:
event_map = rainfall.sel(time=datetime(1996, 9, 3), method="nearest", drop=True)

In [ ]:
event_map.plot()

# 3. Read exposures

In [ ]:
locations = pd.read_csv(COPERNICUS_DATA_PATH / "locations.csv")

In [ ]:
location_map = px.scatter_mapbox(
    locations, lat="Latitude", lon="Longitude", size="Exposure", size_max=100
)
location_map.update_layout(mapbox_style="carto-positron", height=1200)

# 4. Merge exposure to historical data

In [ ]:
rainfall_data = pd.DataFrame(index=pd.Index(rainfall["time"], name="Datetime"))

for _, location in locations.iterrows():
    rainfall_data[location["Address"]] = rainfall.sel(
        lat=location["Latitude"], lon=location["Longitude"], method="nearest", drop=True
    )

In [ ]:
TRIGGER = 80
EXHAUST = 120


def payout_function(yearly_max: pd.Series, exposure: float) -> pd.Series:
    return (
        (yearly_max.clip(lower=TRIGGER, upper=EXHAUST) - TRIGGER)
        / (EXHAUST - TRIGGER)
        * exposure
    )


sample_yearly_max = pd.Series(range(0, 200))
payout_function(sample_yearly_max, exposure=1e5).plot(backend="plotly")

# 5. Task

You have been proposed to cover rainfall exposure for 8 locations in north / east of France. Each location has an exposure that corresponds to its population.\
The cover has the following behavior : 
- We get the index from the data provider that corresponds to the 10-day cummulative rainfall at the nearest grid point. 
- We compute the yearly maximum of this index.
- Depending on this value, we give a payout, proportionnal to the exposure, that depends on the index value.

The payout function has the following caracteristics :
- If lower than 8mm, no payout is received.
- If higher than 16mm, a full payout is triggered (= exposure).
- Between these two values, we compute a linearly interpolated payout.


Your company [choose your nam] has to propose, for each location, a yearly risk premium that you would receive in exchange of the risk. Risk premium are usually defined as a % over the exposure. \
For example, if we charge 10% premium for Lille - with an exposure of 236710 - we get 23671 each year.

The game has the following rules :
1. Among all competing teams, only the one with lowest price will win the contract. It means that this team receive the yearly premium at the begining of the year.
2. In case of equality between some team in rule 1., the priority goes to the team with highest credit at this stage of the game. The choice become random if number of credits is equal.
3. The team that has a contract in a given year has to pay in case of rainfall index higher than 80mm. The amount is defined by the payout function. As the payout is based on yearly maximum index, the team cannot pay more than the exposure every year.
4. In case you don't have enough credit to pay a given claim, you go bankrupt and do not participate in the game anymore. The game continue for the other remaining teams.
5. The game will be performed on different scenarii of 90 simulated years. The team with highest amount of credit at the end wins.

## 6. Strategy outlook

Overview of possible strategies : 

- If you go low pricing, you will win all the contracts but you will in average loose money (if the risk premium is too low). You will go bankrupt soon in the game, hence loose.
- If you're pricing is too high, you will have no contract, and hence no opportunity to win credits.
- If you only select couple of locations to focus on, your portfolio won't be diversified, and you could loose the game in case of bad luck (event on your single location)
- You have to provide a fair pricing, that corresponds to the risk while being lower than your competitors. Ideally, you can select a diversified portfolio and sustained during 90 years.

Please note that no use of external data is allowed.

Generally speaking, there are some chances that an event with return period <100 years will happen during the 90 simulated years. Your company should sustain such an event.

## 7. Delivrable

Each team will have to give a fixed price for the 8 locations based on a statistical analysis of your choice. You can either provide a fixed pricing that will be applied every round, or give a premium that depends on your amount of credit at the beginning of the round.

You will have to explain your reasoning and strategy in a short 2 page document with some metrics justifying your technical choices and limitations.